# StableOps
## Overview

**StableOps** is an analysis of potential opportunities derived from volatility in stablecoin pricing.  It was observed that several stablecoins have some pricing volatility.  Although usually pegged to a FIAT currency, like the US dollar, they sometimes drift to be a little over or under that value.

#TODO: insert histogram and line chart of a stable coin

In theory, if this volatility could be harnessed, it might lead to 4-6% yield.  The objective of this project is to analyze volatility in stablecoin pricing and assess whether this volatility could provide profitable arbitrage opportunities. 

In order to explore this hypothesis further, the following will be analyzed:
* Analysis of the volatility of a sampling of stablecoins based on the US Dollar
    * frequency and magnitude of deviations from pegged stable value
    * statistical distribution
    * line graph
    * histogram
* Analysis of the volatility of a sampling of stablecoins based on a currency other than the US Dollar
    * frequency and magnitude of deviations from pegged stable value
    * statistical distribution
    * line graph
    * histogram
* Analysis of costs associated with exchanging stablecoins with FIAT currency or with another stablecoin
    * resolve a % cost overhead
* Analysis of low risk yield producing options with stable coins, e.g. staking yields
    * resolve an average low risk yield 
* Beta analysis ??? 
    [I seem to remember something about low risk investment opportunity assessment with Beta]
* Analysis of potential arbitrage opportunity
    * Simulations to forecast future returns leveraging this opportunity